# Exploring MRIOs with Pymrio

The first step when working with a new MRIO data set is to familiarize yourself with the data.
This notebook shows how to use the `pymrio` package to explore the data.
We use the test data set that is included in the `pymrio` package.
This is a completely artificial, very small MRIO.
It is not meant to be realistic, but it is useful for developing, testing and learning.

First we import the required packages:

In [1]:
import pymrio

We can now load the test data set with the `load_test` function. We can call
the MRIO whatever we want, here we use mrio.

In [2]:
mrio = pymrio.load_test()

We can get some first information about the MRIO by printing it.

In [3]:
print(mrio)

IO System with parameters: Z, Y, unit, population, meta, factor_inputs, emissions


This tells us what the MRIO data we just loaded contains.
We find a Z and Y matrix, some unit information and two satellite accounts, factor_inputs and emissions.

To get more specific data we can ask pymrio for regions, sectors, products, etc.

In [4]:
mrio.name

'testmrio'

In [5]:
mrio.get_regions()

Index(['reg1', 'reg2', 'reg3', 'reg4', 'reg5', 'reg6'], dtype='object', name='region')

In [6]:
mrio.get_sectors()

Index(['food', 'mining', 'manufactoring', 'electricity', 'construction',
       'trade', 'transport', 'other'],
      dtype='object', name='sector')

In [7]:
mrio.get_Y_categories()

Index(['Final consumption expenditure by households',
       'Final consumption expenditure by non-profit organisations serving households (NPISH)',
       'Final consumption expenditure by government',
       'Gross fixed capital formation', 'Changes in inventories',
       'Changes in valuables', 'Export'],
      dtype='object', name='category')

The same methods can be used to explore one of the satellite accounts.

In [8]:
print(mrio.emissions)

Extension Emissions with parameters: name, F, F_Y, unit


In [9]:
mrio.emissions.name

'Emissions'

In [10]:
mrio.emissions.get_regions()

Index(['reg1', 'reg2', 'reg3', 'reg4', 'reg5', 'reg6'], dtype='object', name='region')

The satellite accounts also have a special method to get index (rows) of the acccounts.

In [11]:
mrio.emissions.get_rows()

MultiIndex([('emission_type1',   'air'),
            ('emission_type2', 'water')],
           names=['stressor', 'compartment'])

# Searching through the MRIO

Several methods are available to search through the whole MRIO.
These generally accept [regular expressions](https://docs.python.org/3/howto/regex.html) as search terms.

The most general method is 'find'. This can be used for a quick overview where a specific term appears in the MRIO.

In [12]:
mrio.find("air")

{'emissions_index': MultiIndex([('emission_type1', 'air')],
            names=['stressor', 'compartment'])}

In [13]:
mrio.find("trade")

{'index': MultiIndex([('reg1', 'trade'),
             ('reg2', 'trade'),
             ('reg3', 'trade'),
             ('reg4', 'trade'),
             ('reg5', 'trade'),
             ('reg6', 'trade')],
            names=['region', 'sector']),
 'sectors': Index(['trade'], dtype='object', name='sector')}

Not that 'find' (and all other search methods) a case sensitive.
Do make a case insensitive search, add the regular expression flag `(?i)` to the search term.

In [14]:
mrio.find("value")

{}

In [15]:
mrio.find("(?i)value")

{'factor_inputs_index': Index(['Value Added'], dtype='object', name='inputtype')}

## Specific search methods: contains, match, fullmatch,

The MRIO class also contains a set of specific regular expresion search methods, mirroring the 'contains', 'match' and 'fullmatch'
methods of the pandas DataFrame str column type. See the pandas documentation for details, in short:

  - 'contains' looks for a match anywhere in the string
  - 'match' looks for a match at the beginning of the string
  - 'fullmatch' looks for a match of the whole string

These methods are available for all index columns of the MRIO and have a similar signature:

  1. As for 'find_all', the search term is case sensitive. To make it case insensitive, add the regular expression flag `(?i)` to the search term.
  2. The search term can be passed to the keyword argument 'find_all' or as the first positional argument to search in all index levels.
  3. Alternativels, the search term can be passed to the keyword argument with the level name to search only in that index level.

The following examples show how to use these methods.

In [16]:
mrio.contains(find_all="ad")
mrio.contains("ad")  # find_all is the default argument

MultiIndex([('reg1', 'trade'),
            ('reg2', 'trade'),
            ('reg3', 'trade'),
            ('reg4', 'trade'),
            ('reg5', 'trade'),
            ('reg6', 'trade')],
           names=['region', 'sector'])

In [17]:
mrio.match("ad")

MultiIndex([], names=['region', 'sector'])

In [18]:
mrio.match("trad")

MultiIndex([('reg1', 'trade'),
            ('reg2', 'trade'),
            ('reg3', 'trade'),
            ('reg4', 'trade'),
            ('reg5', 'trade'),
            ('reg6', 'trade')],
           names=['region', 'sector'])

In [19]:
mrio.fullmatch("trad")

MultiIndex([], names=['region', 'sector'])

In [20]:
mrio.fullmatch("trade")

MultiIndex([('reg1', 'trade'),
            ('reg2', 'trade'),
            ('reg3', 'trade'),
            ('reg4', 'trade'),
            ('reg5', 'trade'),
            ('reg6', 'trade')],
           names=['region', 'sector'])

In [21]:
mrio.fullmatch("(?i).*AD.*")

MultiIndex([('reg1', 'trade'),
            ('reg2', 'trade'),
            ('reg3', 'trade'),
            ('reg4', 'trade'),
            ('reg5', 'trade'),
            ('reg6', 'trade')],
           names=['region', 'sector'])

For the rest of the notebook, we will do the examples with the 'contains' method, but the same applies to the other methods.

To search only at one specific level, pass the search term to the keyword argument with the level name.

In [22]:
mrio.contains(region="trade")

MultiIndex([], names=['region', 'sector'])

In [23]:
mrio.contains(sector="trade")

MultiIndex([('reg1', 'trade'),
            ('reg2', 'trade'),
            ('reg3', 'trade'),
            ('reg4', 'trade'),
            ('reg5', 'trade'),
            ('reg6', 'trade')],
           names=['region', 'sector'])

And of course, the method are also available for the satellite accounts.

In [24]:
mrio.emissions.contains(compartment="air")

MultiIndex([('emission_type1', 'air')],
           names=['stressor', 'compartment'])

Passing a non-existing level to the keyword argument is silently ignored.

In [25]:
mrio.factor_inputs.contains(compartment="trade")

Index([], dtype='object')

This allows to search for terms that are only in some index levels.
Logically, this is an 'or' search.

In [26]:
mrio.factor_inputs.contains(compartment="air", inputtype="Value")

Index(['Value Added'], dtype='object', name='inputtype')

But note, that if both levels exist, both must match (so it becomes a logical 'and').

In [27]:
mrio.emissions.contains(stressor="emission", compartment="air")

MultiIndex([('emission_type1', 'air')],
           names=['stressor', 'compartment'])

## Search through all extensions

All three search methods are also available to loop through all extensions of the MRIO.

In [28]:
mrio.extension_contains(stressor="emission", compartment="air")

{'Factor Inputs': Index([], dtype='object'),
 'Emissions': MultiIndex([('emission_type1', 'air')],
            names=['stressor', 'compartment'])}

If only a subset of extensions should be searched, pass the extension names to the keyword argument 'extensions'.

## Generic search method for any dataframe index

Internally, the class methods 'contains', 'match' and 'fullmatch' all the
'index_contains', 'index_match' and 'index_fullmatch' methods of ioutil module.
This function can be used to search through index of any pandas DataFrame.

In [29]:
df = mrio.Y

Depending if a dataframe or an index is passed, the return is either the dataframe or the index.

In [30]:
pymrio.index_contains(df, "trade")

region                                               reg1  \
category      Final consumption expenditure by households   
region sector                                               
reg1   trade                                 769535.93000   
reg2   trade                                   5678.26740   
reg3   trade                                   2753.86080   
reg4   trade                                    373.28393   
reg5   trade                                   4287.40670   
reg6   trade                                   4772.75750   

region                                                                                              \
category      Final consumption expenditure by non-profit organisations serving households (NPISH)   
region sector                                                                                        
reg1   trade                                           16.638920                                     
reg2   trade                                            0.075424                                     
reg3   trade                                            0.111540                                     
reg4   trade                                            0.009382                                     
reg5   trade                                            0.038941                                     
reg6   trade                                            0.113112                                     

region                                                     \
category      Final consumption expenditure by government   
region sector                                               
reg1   trade                                 2.408807e+07   
reg2   trade                                 2.312962e+02   
reg3   trade                                 1.956911e+00   
reg4   trade                                 3.585011e-01   
reg5   trade                                 7.014679e+00   
reg6   trade                                 2.321101e+01   

region                                                              \
category      Gross fixed capital formation Changes in inventories   
region sector                                                        
reg1   trade                   6.727345e+07            1230.218200   
reg2   trade                   6.339521e+02              35.607157   
reg3   trade                   3.598675e+02              23.391120   
reg4   trade                   2.514957e-02               0.002016   
reg5   trade                   1.955479e+02               6.675656   
reg6   trade                   2.417571e+02              16.267049   

region                                     \
category      Changes in valuables Export   
region sector                               
reg1   trade            216.211080      0   
reg2   trade              3.192694      0   
reg3   trade              0.000455      0   
reg4   trade              0.000144      0   
reg5   trade              0.524015      0   
reg6   trade              1.488818      0   

region                                               reg2  \
category      Final consumption expenditure by households   
region sector                                               
reg1   trade                                   8063.52380   
reg2   trade                                 385664.01000   
reg3   trade                                   2072.24890   
reg4   trade                                    192.21539   
reg5   trade                                   3633.68750   
reg6   trade                                   2031.49640   

region                                                                                              \
category      Final consumption expenditure by non-profit organisations serving households (NPISH)   
region sector                                                                                        
reg1   trade                                           12.738233                        

In [31]:
pymrio.index_contains(df.index, "trade")

MultiIndex([('reg1', 'trade'),
            ('reg2', 'trade'),
            ('reg3', 'trade'),
            ('reg4', 'trade'),
            ('reg5', 'trade'),
            ('reg6', 'trade')],
           names=['region', 'sector'])

In [32]:
pymrio.index_fullmatch(df, region="reg[2,4]", sector="m.*")

region                                                      reg1  \
category             Final consumption expenditure by households   
region sector                                                      
reg2   mining                                       1.653997e+02   
       manufactoring                                9.928459e+07   
reg4   mining                                       1.072728e+02   
       manufactoring                                4.086352e+07   

region                                                                                                     \
category             Final consumption expenditure by non-profit organisations serving households (NPISH)   
region sector                                                                                               
reg2   mining                                              1.817989e-05                                     
       manufactoring                                       4.187143e+00                                     
reg4   mining                                              9.421644e-09                                     
       manufactoring                                       1.170611e+00                                     

region                                                            \
category             Final consumption expenditure by government   
region sector                                                      
reg2   mining                                           0.334824   
       manufactoring                                 1373.370200   
reg4   mining                                           0.209851   
       manufactoring                                  377.322810   

region                                                                     \
category             Gross fixed capital formation Changes in inventories   
region sector                                                               
reg2   mining                         3.283238e+01           2.910648e+01   
       manufactoring                  4.237878e+07           4.415752e+03   
reg4   mining                         1.055704e+00           2.697312e+01   
       manufactoring                  3.032655e+07           2.263532e+06   

region                                            \
category             Changes in valuables Export   
region sector                                      
reg2   mining                3.970468e-05      0   
       manufactoring         1.637658e+02      0   
reg4   mining                3.112643e-09      0   
       manufactoring         4.696135e+01      0   

region                                                      reg2  \
category             Final consumption expenditure by households   
region sector                                                      
reg2   mining                                       1.091126e+03   
       manufactoring                                3.210316e+05   
reg4   mining                                       2.940734e+02   
       manufactoring                                1.510445e+07   

region                                                                                                     \
category             Final consumption expenditure by non-profit organisations serving households (NPISH)   
region sector                                                                                               
reg2   mining                                                  2.751312                                     
       manufactoring                                         125.729110                                     
reg4   mining                                                  0.000001                                     
       manufactoring                                           7.598440                                     

region                                                            ...  \
category             Final consumption expenditure by governmen

All search methods can easily be combined with the extract methods to extract the data that was found.
For more information on this, see the [extract_data](./extract_data.ipynb) notebook.